In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
import pandas as pd
import random
import seaborn as sns
sns.set_context('talk')

**The following code uses the exponential to recover a value with a given noise**

This means that if the value in the exponential is 0.8, it will return values around 0.8. This is not what we have in behavior, were the values themselves come fro a binomial distribution and we average them to recover a mean

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Define the true exponential decay function with saturation
def true_function(x, k):
    return 0.5 + (1 - 0.5) * np.exp(-k * x)

# Define the model function for fitting
def model_function(x, a, b, c):
    return a + (b - a) * np.exp(-c * x)

# Generate true data
x_true = np.linspace(0, 10, 100)
k_true = 0.05
y_true = true_function(x_true, k_true)

# Define noise level
noise_level = 0.05

# scenario 1: 4 discrete points with 100 repetitions each
x_scenario1 = np.linspace(0, 10, 4)
data_scenario1 = []

for x in x_scenario1:
    for _ in range(100):
        y_noisy = true_function(x, k_true) + np.random.normal(0, noise_level)
        data_scenario1.append((x, y_noisy))

data_scenario1 = np.array(data_scenario1)

# Fit curve for scenario 1
params_scenario1, _ = curve_fit(model_function, data_scenario1[:, 0], data_scenario1[:, 1], p0=[0.5, 1, 0.5])

# scenario 2: 100 discrete points with 4 repetitions each
x_scenario2 = np.linspace(0, 10, 100)
data_scenario2 = []

for x in x_scenario2:
    for _ in range(4):
        y_noisy = true_function(x, k_true) + np.random.normal(0, noise_level)
        data_scenario2.append((x, y_noisy))

data_scenario2 = np.array(data_scenario2)

# Fit curve for scenario 2
params_scenario2, _ = curve_fit(model_function, data_scenario2[:, 0], data_scenario2[:, 1], p0=[0.5, 1, 0.5])

# Plot the results
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(data_scenario1[:, 0], data_scenario1[:, 1], color='blue', label='Noisy Data (scenario 1)')
x_fit = np.linspace(0, 10, 100)
plt.plot(x_fit, model_function(x_fit, *params_scenario1), color='red', label='Fitted Curve (scenario 1)')
plt.title('scenario 1')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(data_scenario2[:, 0], data_scenario2[:, 1], color='green', label='Noisy Data (scenario 2)')
plt.plot(x_fit, model_function(x_fit, *params_scenario2), color='orange', label='Fitted Curve (scenario 2)')
plt.title('scenario 2')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()

plt.tight_layout()
plt.show()

print("True values of the curve:")
print(f"Saturation value: 0.5")
print(f"Maximum value: 1.0")
print(f"Decay rate (c): {k_true:.3f}")

print("\nFitted parameters for scenario 1:")
print(f"a: {params_scenario1[0]:.3f}, b: {params_scenario1[1]:.3f}, c: {params_scenario1[2]:.3f}")

print("\nFitted parameters for scenario 2:")
print(f"a: {params_scenario2[0]:.3f}, b: {params_scenario2[1]:.3f}, c: {params_scenario2[2]:.3f}")




**This code does this simulation with prbability distributions that decay in time**

In [ ]:
def simulate_fit(k_true: float=0.02, 
                 noise_level: float=0.01, 
                 model_type: str= 'exponential', 
                 iteration = 0,
                 results_df: pd.DataFrame = pd.DataFrame(columns=['scenario', 'model_type', 'mse', 'r_squared', 'param_a', 'param_b', 'param_c', 'param_m', 'param_b_linear'])
):

    # Define the true exponential decay function for probabilities
    def true_function(x, k, model_type: str = 'exponential'):
        if model_type == 'exponential':
            return 0.5 + 0.5 * np.exp(-k * x)
        if model_type == 'linear':
            return -k * x + 1

    # Define the exponential decay model function
    def exp_model(x, a, b, c):
        return a + b * np.exp(-c * x)

    # Define the linear model function
    def linear_model(x, m, b):
        return -m * x + b

    # Function to fit the model based on the chosen type
    def fit_curve(x, y, model_type):
        if model_type == 'exponential':
            params, _ = curve_fit(exp_model, x, y, p0=[0.5, 0.5, 0.5])
        elif model_type == 'linear':
            params, _ = curve_fit(linear_model, x, y, p0=[0.5, 0.5])
        else:
            raise ValueError("Unsupported model_type. Choose 'exponential' or 'linear'.")
        return params

    # Function to compute r_squared
    def r_squared(y_true, y_pred):
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        return 1 - (ss_res / ss_tot)

    # Generate true data
    x_true = np.linspace(0, 10, 100)
    y_true = true_function(x_true, k_true, model_type)

    # Initialize a DataFrame to store results
    results_df = pd.DataFrame(columns=['scenario', 'model_type', 'mse', 'r_squared', 'param_a', 'param_b', 'param_c', 'param_m', 'param_b_linear'])

    # scenario 1: 4 discrete points with 100 repetitions each
    x_scenario1 = np.array([0.1, 1, 3, 10])
    data_scenario1 = []

    for _ in range(400):
        # for x in x_scenario1:
        x = random.choice(x_scenario1)
        probability = true_function(x, k_true, model_type)
        binary_outcome = np.random.binomial(1, probability)
        data_scenario1.append((x, binary_outcome))

    data_scenario1 = np.array(data_scenario1)

    # Compute mean of binary outcomes for each x-value in scenario 1
    mean_scenario1 = [np.mean(data_scenario1[data_scenario1[:, 0] == x, 1]) for x in x_scenario1]

    # Fit curve for scenario 1
    params_scenario1 = fit_curve(x_scenario1, mean_scenario1, model_type)

    # Compute predictions for the x-values used in scenario 1
    if model_type == 'exponential':
        predictions_scenario1 = exp_model(x_scenario1, *params_scenario1)
        param_a, param_b, param_c = params_scenario1
        param_m, param_b_linear = np.nan, np.nan
    elif model_type == 'linear':
        predictions_scenario1 = linear_model(x_scenario1, *params_scenario1)
        param_a, param_b, param_c = np.nan, np.nan, np.nan
        param_m, param_b_linear = params_scenario1

    # Compute mse and r_squared for scenario 1
    mse_scenario1 = mean_squared_error(true_function(x_scenario1, k_true), predictions_scenario1)
    r2_scenario1 = r_squared(true_function(x_scenario1, k_true), predictions_scenario1)

    # Append results to DataFrame
    results_df = results_df.append({
        'iteration' : iteration,
        'scenario': 'Discrete',
        'model_type': model_type,
        'mse': mse_scenario1,
        'r_squared': r2_scenario1,
        'param_a': param_a,
        'param_b': param_b,
        'param_c': param_c,
        'param_m': param_m,
        'param_b_linear': param_b_linear,
        'real_fit' : k_true
    }, ignore_index=True)

    # scenario 2: 100 discrete points with 4 repetitions each
    x_scenario2 = np.linspace(0, 10, 100)
    data_scenario2 = []
    
    for _ in range(400):
        x = random.choice(x_scenario2)
        # for x in x_scenario2:
        probability = true_function(x, k_true, model_type)
        binary_outcome = np.random.binomial(1, probability)
        data_scenario2.append((x, binary_outcome))
    data_scenario2 = np.array(data_scenario2)

    # Compute mean of binary outcomes for each x-value in scenario 2
    mean_scenario2 = [np.mean(data_scenario2[data_scenario2[:, 0] == x, 1]) for x in x_scenario2]

    # Create a boolean mask where mean_scenario is not NaN
    mask = ~np.isnan(mean_scenario2)
    x_scenario2 = x_scenario2[mask]
    mean_scenario2 = np.array(mean_scenario2)
    mean_scenario2 = mean_scenario2[mask]

    # Fit curve for scenario 2
    params_scenario2 = fit_curve(x_scenario2, mean_scenario2, model_type)

    # Compute predictions for the x-values used in scenario 2
    if model_type == 'exponential':
        predictions_scenario2 = exp_model(x_scenario2, *params_scenario2)
        param_a, param_b, param_c = params_scenario2
        param_m, param_b_linear = np.nan, np.nan
    elif model_type == 'linear':
        predictions_scenario2 = linear_model(x_scenario2, *params_scenario2)
        param_a, param_b, param_c = np.nan, np.nan, np.nan
        param_m, param_b_linear = params_scenario2

    # Compute mse and r_squared for scenario 2
    mse_scenario2 = mean_squared_error(true_function(x_scenario2, k_true), predictions_scenario2)
    r2_scenario2 = r_squared(true_function(x_scenario2, k_true), predictions_scenario2)

    # Append results to DataFrame
    results_df = results_df.append({
        'iteration' : iteration,
        'scenario': 'Uniform',
        'model_type': model_type,
        'mse': mse_scenario2,
        'r_squared': r2_scenario2,
        'param_a': param_a,
        'param_b': param_b,
        'param_c': param_c,
        'param_m': param_m,
        'param_b_linear': param_b_linear,
        'real_fit' : k_true
    }, ignore_index=True)

    # Optionally, save the DataFrame to a CSV file
    results_df.to_csv('model_fit_results.csv', index=False)

    # Plot the results
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(x_true, y_true, color='black', label='True Data')
    plt.plot(x_scenario1, mean_scenario1, 'o', color='blue', label='Average Data (scenario 1)')
    x_fit = np.linspace(0, 10, 100)
    if model_type == 'exponential':
        plt.plot(x_fit, exp_model(x_fit, *params_scenario1), color='red', label='Fitted Curve (scenario 1)')
    elif model_type == 'linear':
        plt.plot(x_fit, linear_model(x_fit, *params_scenario1), color='red', label='Fitted Curve (scenario 1)')
    plt.title('Discrete')
    plt.xlabel('Delay (s)')
    plt.ylim(0.45, 1.05)
    plt.ylabel('Average Probability')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(x_true, y_true, color='black', label='True Data')
    plt.plot(x_scenario2, mean_scenario2, 'o', color='green', label='Average Data (scenario 2)')
    if model_type == 'exponential':
        plt.plot(x_fit, exp_model(x_fit, *params_scenario2), color='orange', label='Fitted Curve (scenario 2)')
    elif model_type == 'linear':
        plt.plot(x_fit, linear_model(x_fit, *params_scenario2), color='orange', label='Fitted Curve (scenario 2)')
    plt.title('Uniform')
    plt.xlabel('Delay (s)')
    plt.ylim(0.45, 1.05)
    plt.ylabel('Average Probability')
    plt.legend()

    plt.tight_layout()
    plt.show()

    return results_df


In [ ]:
df = simulate_fit(k_true=0.01, model_type='linear')
print(df)

In [ ]:
df_all = pd.DataFrame()
i=0
while i <100:
    try:
        df = simulate_fit(k_true=0.05, model_type='exponential')
        df_all = pd.concat([df_all, df], ignore_index=True)
        i+=1
        print(i)
    except:
        continue

In [ ]:
i=0
while i <100:
    try:
        df = simulate_fit(k_true=0.01, model_type='linear')
        df_all = pd.concat([df_all, df], ignore_index=True)
        i+=1
    except:
        continue

In [ ]:
df_all.to_csv('simulation_discrete_continuous_delays.csv')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,5))
sns.boxplot(x='scenario', y='mse', data = df_all.loc[df_all.model_type == 'exponential'], width=0.5, ax= ax[0])
sns.boxplot(x='scenario', y='r_squared', data = df_all.loc[df_all.model_type == 'exponential'], width=0.5, ax= ax[1])
sns.boxplot(x='scenario', y='param_c', data = df_all.loc[df_all.model_type == 'exponential'], width=0.5, ax= ax[2])
sns.scatterplot(x='scenario', y='real_fit', data = df_all.loc[df_all.model_type == 'exponential'], color='purple', ax= ax[2])
plt.suptitle('Exponential')
sns.despine()
plt.tight_layout()


In [ ]:
fig, ax = plt.subplots(1,3, figsize=(14,5))
sns.boxplot(x='scenario', y='mse', data = df_all.loc[df_all.model_type == 'linear'], width=0.5, ax= ax[0])
sns.boxplot(x='scenario', y='r_squared', data = df_all.loc[df_all.model_type == 'linear'], width=0.5, ax= ax[1])
sns.boxplot(x='scenario', y='param_m', data = df_all.loc[df_all.model_type == 'linear'], width=0.5, ax= ax[2])
sns.scatterplot(x='scenario', y='real_fit', data = df_all.loc[df_all.model_type == 'linear'], color='purple', ax= ax[2])
plt.suptitle('Linear')
sns.despine()
plt.tight_layout()
